In [ ]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import imageio
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
mpl.rcParams['figure.dpi'] = 500

class Agent:
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color  # 'red' or 'blue'

def move_agents(agents, speed_map, width, height, change_map):
    interaction_distance = 10  # Interaction distance
    for agent in agents:
        if np.isnan(speed_map[int(agent.y), int(agent.x)]):
            continue  # Skip movement if on NaN area
        
        speed = speed_map[int(agent.y), int(agent.x)]
        angle = np.random.uniform(0, 2 * np.pi)
        dx = np.cos(angle) * speed
        dy = np.sin(angle) * speed

        new_x = agent.x + dx
        new_y = agent.y + dy

        if new_x < 0:
            agent.x = -new_x
        elif new_x >= width:
            agent.x = 2 * width - new_x - 1
        else:
            agent.x = new_x

        if new_y < 0:
            agent.y = -new_y
        elif new_y >= height:
            agent.y = 2 * height - new_y - 1
        else:
            agent.y = new_y
        
        for other in agents:
            if other != agent and np.hypot(agent.x - other.x, agent.y - other.y) < interaction_distance:
                if agent.color != other.color and np.random.random() < 0.5:
                    change_map[int(agent.y), int(agent.x)] += 1
                    agent.color = other.color

speed_map_path = '/Users/braydennoh/Downloads/speed_map.tif'
with rasterio.open(speed_map_path) as src:
    speed_map = src.read(1)
    width, height = src.width, src.height

num_agents = 1000
agents = []
colors = ['red', 'blue'] * (num_agents // 2)
np.random.shuffle(colors)
buffer = 10

change_map = np.zeros((height, width))

while len(agents) < num_agents:
    x = np.random.randint(buffer, width - buffer)
    y = np.random.randint(buffer, height - buffer)
    if not np.isnan(speed_map[y, x]):
        agents.append(Agent(x, y, colors.pop()))

num_iterations = 500

for _ in range(num_iterations):
    move_agents(agents, speed_map, width, height, change_map)
    
plt.figure(figsize=(3, 3))
plt.axis("off")
plt.imshow(change_map, cmap='hot', interpolation='nearest')